In [1]:
import ROOT as R
import sys
import os
import uproot


In [2]:
files = os.listdir()
root_files = []
print("you have the following files to analyse")
for i, f in enumerate(files):
	if f[-4:] == "root":
		print("file n. ", i , " file" , f)
		root_files.append(f)
l = len(files)


you have the following files to analyse
file n.  3  file KM3NeT_00000254_00003378.root
file n.  4  file KM3NeT_00000254_00003382.root
file n.  5  file KM3NeT_00000254_00003382__HVTUNED_L1.JRA.root


In [ ]:
n = False
count = 0

while True:
	count +=1
	if count >5:
		print("Come on man, you can do better than this")
		print("I'm outta here")
		break
	n = int(input("Insert the number of file you want to analyse"))
	if type(n) == int:
		
		if n > l-1:
			print("out of range!")
		else:
			print("File selection is valid")
			fname = files[n]
			break
	else:
		print("This is not an integer")

File selection is valid


In [4]:
fname

'First_analysis.ipynb'

In [74]:
file = R.TFile.Open(fname, "READ")
if not file or file.IsZombie():
    print("Qualcosa non va con i file (corrotti/mancanti)")
    sys.exit(1)
key_list = file.GetListOfKeys()
for key in key_list:
    print(key.GetName())

META
KM3NETDAQ::JDAQHeader
Detector
JDAQEvent
RUNINFO


In [75]:
detector_dir = file.Get("Detector")
if not detector_dir:
    print("Non riesco ad accedere alla cartella")
else:
    key_list = detector_dir.GetListOfKeys()
    print("Oggetti contenuti in 'Detector':")
    for key in key_list:
        print("name: ", key.GetName(), "- Class: ", key.GetClassName())


Oggetti contenuti in 'Detector':
name:  h_fifo_per_dom - Class:  TH2D
name:  h_daq_status_per_dom - Class:  TH2D
name:  h_hrv_per_dom - Class:  TH2D
name:  h_rate_summary - Class:  TH2D
name:  h_pmt_rate_distribution - Class:  TH1D
name:  h_dom_rate_distribution - Class:  TH1D
name:  h_mean_dom_rates_Timeslice - Class:  TH2D
name:  h_mean_dom_rates_TimesliceL1 - Class:  TH2D
name:  h_mean_dom_rates_TimesliceSN - Class:  TH2D
name:  DU1 - Class:  TDirectoryFile


In [76]:
def hist_plot_2d(file,path, object, outname= ""): #if the object fed is a 2d histogram, this shows and saves it with the given name
#path must not have in it the last "/" e.g. Desktop/files/analysis
#in the path, you should put JUST THE PATH BEFORE THE FOLDER, WITHOUT the folder included
    obj_class = object.GetClassName()
    name =object.GetName()

    if obj_class== "TH2D":
        
        if 'canvas' in globals():
            canvas.Close()
        file = R.TFile.Open(fname, "READ")
        if not file or file.IsZombie():
            print("Qualcosa non va con i file (corrotti/mancanti)")
            sys.exit(1)
        
            
        h_2d = file.Get(path+"/"+name)
        if 'canvas' in globals():
            canvas.Close()
        print(path+"/"+name)
        # Crea un canvas 
        canvas = R.TCanvas("firstplot",  '',  2400, 800) #la prima stringa è il nome del canvas. La seconda non ne ho idea, ma non serve. La terza e la quarta sono le dimensioni

        R.gStyle.SetOptStat(0)

        h_2d.Draw("COLZ")
        #canvas = R.TCanvas("firstplot",  '',  2400, 800) #la prima stringa è il nome del canvas. La seconda non ne ho idea, ma non serve. La terza e la quarta sono le dimensioni

        
        if outname == "":
            print("Drawing" + name)
            canvas.SaveAs(path+"/"+name+".png")
        else:
            print("Drawing" + outname)
            canvas.SaveAs(outname+".png")
        canvas.Show()
    return None

In [114]:
def all_2d_in_folder(fname, path, folder):
    folders = []
    file = R.TFile.Open(fname, "READ")
    detector_dir = file.Get(path + folder)
    print("opening folder ",path, folder , " ...")
    if not detector_dir:
        print("Non riesco ad accedere alla cartella", path,  folder)
    else:
        key_list = detector_dir.GetListOfKeys()
        print("Oggetti contenuti in ", folder)
        for key in key_list:
            if key.GetClassName() == "TDirectoryFile":
                folders.append(key.GetName())
            hist_plot_2d(file,path, key)
    return folders


In [118]:
subfolders = all_2d_in_folder(fname,"","Detector")

opening folder   Detector  ...
Oggetti contenuti in  Detector
/h_fifo_per_dom


ReferenceError: attempt to access a null-pointer

Warning in <TCanvas::Constructor>: Deleting canvas with same name: firstplot


In [110]:
subfolders = all_2d_in_folder(fname,"Detector", "DU1")

opening folder  Detector / DU1  ...
Oggetti contenuti in  DU1
Detector/h_fifo_per_dom
Drawingh_fifo_per_dom
Detector/h_daq_status_per_dom
Drawingh_daq_status_per_dom
Detector/h_hrv_per_dom
Drawingh_hrv_per_dom
Detector/h_rate_summary
Drawingh_rate_summary
Detector/h_mean_dom_rates_Timeslice
Drawingh_mean_dom_rates_Timeslice
Detector/h_mean_dom_rates_TimesliceL1
Drawingh_mean_dom_rates_TimesliceL1
Detector/h_mean_dom_rates_TimesliceSN
Drawingh_mean_dom_rates_TimesliceSN


Info in <TCanvas::Print>: png file Detector/h_fifo_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_daq_status_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_hrv_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_rate_summary.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_Timeslice.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_TimesliceL1.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_TimesliceSN.png has been created


In [100]:
subfolder = "Detector/DU1"
subfolders

['DU1']

In [103]:
more_subfolder = all_2d_in_folder(fname, "Detector", "DU1")


Oggetti contenuti in 'Detector':
Detector/h_fifo_per_dom
Drawingh_fifo_per_dom
Detector/h_daq_status_per_dom
Drawingh_daq_status_per_dom
Detector/h_hrv_per_dom
Drawingh_hrv_per_dom
Detector/h_rate_summary
Drawingh_rate_summary
Detector/h_mean_dom_rates_Timeslice
Drawingh_mean_dom_rates_Timeslice
Detector/h_mean_dom_rates_TimesliceL1
Drawingh_mean_dom_rates_TimesliceL1
Detector/h_mean_dom_rates_TimesliceSN
Drawingh_mean_dom_rates_TimesliceSN


Info in <TCanvas::Print>: png file Detector/h_fifo_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_daq_status_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_hrv_per_dom.png has been created
Info in <TCanvas::Print>: png file Detector/h_rate_summary.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_Timeslice.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_TimesliceL1.png has been created
Info in <TCanvas::Print>: png file Detector/h_mean_dom_rates_TimesliceSN.png has been created


In [104]:
more_subfolder

['DU1']

In [101]:
detector_dir = file.Get(subfolder)
if not detector_dir:
    print("Non riesco ad accedere alla cartella")
else:
    key_list = detector_dir.GetListOfKeys()
    print("Oggetti contenuti in 'Detector':")
    for key in key_list:
        print("name: ", key.GetName(), "- Class: ", key.GetClassName())


Oggetti contenuti in 'Detector':
name:  F1 - Class:  TDirectoryFile
name:  F10 - Class:  TDirectoryFile
name:  F11 - Class:  TDirectoryFile
name:  F12 - Class:  TDirectoryFile
name:  F13 - Class:  TDirectoryFile
name:  F14 - Class:  TDirectoryFile
name:  F15 - Class:  TDirectoryFile
name:  F16 - Class:  TDirectoryFile
name:  F17 - Class:  TDirectoryFile
name:  F18 - Class:  TDirectoryFile
name:  F2 - Class:  TDirectoryFile
name:  F3 - Class:  TDirectoryFile
name:  F4 - Class:  TDirectoryFile
name:  F5 - Class:  TDirectoryFile
name:  F6 - Class:  TDirectoryFile
name:  F7 - Class:  TDirectoryFile
name:  F8 - Class:  TDirectoryFile
name:  F9 - Class:  TDirectoryFile
name:  h_mean_summary_rate - Class:  TH2D
name:  h_mean_summary_rate_distribution - Class:  TH1D
name:  h_mean_ToT_Timeslice - Class:  TH2D
name:  h_mean_ToT_TimesliceL1 - Class:  TH2D
name:  h_mean_ToT_TimesliceSN - Class:  TH2D
name:  h_mean_ToT_distributionTimeslice - Class:  TH1D
name:  h_mean_ToT_distributionTimesliceL1 -

In [60]:
def hist_plot_2d(object, outname= ""): #if the object fed is a 2d histogram, this shows and saves it with the given name

    obj_class = object.GetClassName()
    name =object.GetName()

    if obj_class== "TH2D":
        if 'canvas' in globals():
            canvas.Close()
        file = R.TFile.Open(fname, "READ")
        if not file or file.IsZombie():
            print("Qualcosa non va con i file (corrotti/mancanti)")
            sys.exit(1)
        
            
        h_2d = file.Get(subfolder+"/" + name)
        if 'canvas' in globals():
            canvas.Close()
        # Crea un canvas 
        canvas = R.TCanvas("firstplot",  '',  2400, 800) #la prima stringa è il nome del canvas. La seconda non ne ho idea, ma non serve. La terza e la quarta sono le dimensioni

        R.gStyle.SetOptStat(0)

        h_2d.Draw("COLZ")
        #canvas = R.TCanvas("firstplot",  '',  2400, 800) #la prima stringa è il nome del canvas. La seconda non ne ho idea, ma non serve. La terza e la quarta sono le dimensioni

        
        if outname == "":
            print("Drawing" + name)
            canvas.SaveAs(name+".png")
        else:
            print("Drawing" + outname)
            canvas.SaveAs(outname+".png")
        canvas.Show()
    return None

In [61]:

def dO_all_subfolders(function, file,subfolder, max_iter = 100 ):
    
    #this plots using the fed function, all the files in a fed folder, and in its subfolder. By fed i mean input. 
    # Please mutherfucker be careful that your gucking function does NOT break if you give it a wrong type.
    #just make it ignore the object with the worng type, deal?
    #it has a number of maximum iteration in order to avoid when there are too many folders to get stuck
    iter = 0 

    detector_dir = file.Get(subfolder.GetName())
    
    key_list = detector_dir.GetListOfKeys()

    for key in key_list:
    
        

        if iter> max_iter:
            break
        else:
            obj_class = key.GetClassName()

            if obj_class == "TDirectoryFile":
                #dO_all_folder(function, key)
                iter+=1
            else: 
                iter+=1
                function(key)     

    return None

In [62]:
detector_dir = file.Get(subfolder)
if not detector_dir:
    print("Non riesco ad accedere alla cartella")
else:
    key_list = detector_dir.GetListOfKeys()
    print("Oggetti contenuti in 'Detector':")
    for key in key_list:
        print("name: ", key.GetName(), "- Class: ", key.GetClassName())
        hist_plot_2d(key)


Oggetti contenuti in 'Detector':
name:  F1 - Class:  TDirectoryFile
name:  F10 - Class:  TDirectoryFile
name:  F11 - Class:  TDirectoryFile
name:  F12 - Class:  TDirectoryFile
name:  F13 - Class:  TDirectoryFile
name:  F14 - Class:  TDirectoryFile
name:  F15 - Class:  TDirectoryFile
name:  F16 - Class:  TDirectoryFile
name:  F17 - Class:  TDirectoryFile
name:  F18 - Class:  TDirectoryFile
name:  F2 - Class:  TDirectoryFile
name:  F3 - Class:  TDirectoryFile
name:  F4 - Class:  TDirectoryFile
name:  F5 - Class:  TDirectoryFile
name:  F6 - Class:  TDirectoryFile
name:  F7 - Class:  TDirectoryFile
name:  F8 - Class:  TDirectoryFile
name:  F9 - Class:  TDirectoryFile
name:  h_mean_summary_rate - Class:  TH2D
Drawingh_mean_summary_rate
name:  h_mean_summary_rate_distribution - Class:  TH1D
name:  h_mean_ToT_Timeslice - Class:  TH2D
Drawingh_mean_ToT_Timeslice
name:  h_mean_ToT_TimesliceL1 - Class:  TH2D
Drawingh_mean_ToT_TimesliceL1
name:  h_mean_ToT_TimesliceSN - Class:  TH2D
Drawingh_mean

Info in <TCanvas::Print>: png file h_mean_summary_rate.png has been created
Info in <TCanvas::Print>: png file h_mean_ToT_Timeslice.png has been created
Info in <TCanvas::Print>: png file h_mean_ToT_TimesliceL1.png has been created
Info in <TCanvas::Print>: png file h_mean_ToT_TimesliceSN.png has been created
Info in <TCanvas::Print>: png file h_Snapshot_hits_per_pmt.png has been created


In [63]:
subfolder +="/F1"
detector_dir = file.Get(subfolder)
if not detector_dir:
    print("Non riesco ad accedere alla cartella")
else:
    key_list = detector_dir.GetListOfKeys()
    print("Oggetti contenuti in 'Detector':")
    for key in key_list:
        print("name: ", key.GetName(), "- Class: ", key.GetClassName())

Oggetti contenuti in 'Detector':
name:  805695432_Timeslice_2SToT - Class:  TH2D
name:  805695432_TimesliceL1_2SToT - Class:  TH2D
name:  805695432_TimesliceSN_2SToT - Class:  TH2D
name:  h_pmt_rate_distributions_Timeslice - Class:  TH2D
name:  h_pmt_rate_distributions_TimesliceL1 - Class:  TH2D
name:  h_pmt_rate_distributions_TimesliceSN - Class:  TH2D
name:  h_pmt_rate_distributions_Summaryslice - Class:  TH2D


In [64]:
file = R.TFile.Open(fname, "READ")
if not file or file.IsZombie():
    print("Qualcosa non va con i file (corrotti/mancanti)")
    sys.exit(1)
key_list = file.GetListOfKeys()
for key in key_list:
    key.Print()

TKey Name = META, Title = META, Cycle = 1
TKey Name = KM3NETDAQ::JDAQHeader, Title = object title, Cycle = 1
TKey Name = Detector, Title = Detector, Cycle = 1
TKey Name = JDAQEvent, Title = JDAQEvent, Cycle = 1
TKey Name = RUNINFO, Title = run_info, Cycle = 1


In [65]:
file = R.TFile.Open(fname, "READ")
if not file or file.IsZombie():
    print("Qualcosa non va con i file (corrotti/mancanti)")
    sys.exit(1)
key_list = file.GetListOfKeys()
for key in key_list:
    hist_plot_2d(key)

In [67]:
def do_all_subfolders_2(function, directory, depth=0, max_depth=5):
    """
    Funzione ricorsiva che esplora una directory ROOT e applica una funzione a tutti gli istogrammi.
    """
    if depth > max_depth:
        print(f"Raggiunta profondità massima in {directory.GetName()}")
        return

    key_list = directory.GetListOfKeys()
    for key in key_list:
        obj_name = key.GetName()
        obj_class = key.GetClassName()

        if obj_class == "TDirectoryFile":
            subdir = directory.Get(obj_name)
            if subdir:
                do_all_subfolders(function, subdir, depth+1, max_depth)
        elif obj_class.startswith("TH"):  # istogramma TH1*, TH2*, TH3*
            try:
                obj = directory.Get(obj_name)
                if obj:
                    function(obj, obj_name)
            except Exception as e:    
                detector_dir = file.Get(subfolder.GetName())

                print(f"Errore su {obj_name}: {e}")
        else:
            # ignora oggetti non riconosciuti
            print(f"Ignoro oggetto {obj_name} di tipo {obj_class}")


In [68]:
file = R.TFile.Open(fname, "READ")
if not file or file.IsZombie():
    print("File corrotto o mancante")
    sys.exit(1)

do_all_subfolders(hist_plot_2d, file)


NameError: name 'do_all_subfolders' is not defined

In [15]:
detector_dir = file.Get(subfolder)

key_list = detector_dir.GetListOfKeys()

for hist in key_list:
    







IndentationError: expected an indented block (2376437985.py, line 11)

In [15]:
for key in key_list:
    name = key.GetName()
    class_name = key.GetClassName()
    print(name, class_name)
    
    # Salta se non è un istogramma
    if not class_name.startswith("TH"):
        continue

    print(f"Sto aprendo {name} di tipo {class_name}")

    if 'canvas' in globals():
        canvas.Close()

    # Riapri il file per evitare problemi di riferimento
    file = R.TFile.Open(fname, "READ")
    h_obj = file.Get(subfolder + "/" + name)

    if not h_obj:
        print(f"Non riesco a caricare {name}")
        continue

    canvas = R.TCanvas("canvas", "", 2400, 800)
    R.gStyle.SetOptStat(0)
    h_obj.Draw("COLZ" if class_name.startswith("TH2") else "")
    canvas.SaveAs(f"{name}.png")


TypeError: none of the 3 overloaded methods succeeded. Full details:
  TIter::TIter(const TCollection* col, Bool_t dir = kIterForward) =>
    SegmentationViolation: segfault in C++; program state was reset
  TIter::TIter(TIterator* it) =>
    TypeError: could not convert argument 1
  TIter::TIter(const TIter& iter) =>
    TypeError: could not convert argument 1

 *** Break *** segmentation violation



There was a crash.
This is the entire stack trace of all threads:

Thread 30 (Thread 0x7372907f66c0 (LWP 30687) "ZMQbg/Reaper"):
#0  0x000073729352a042 in epoll_wait (epfd=6, events=0x7372907f51d0, maxevents=256, timeout=-1) at ../sysdeps/unix/sysv/linux/epoll_wait.c:30
#1  0x0000737293066f59 in zmq::epoll_t::loop() () from /home/lori/Downloads/home/anaconda3/envs/root_env/lib/python3.9/site-packages/zmq/backend/cython/../../../../../libzmq.so.5
#2  0x00007372930a22bf in thread_routine () from /home/lori/Downloads/home/anaconda3/envs/root_env/lib/python3.9/site-packages/zmq/backend/cython/../../../../../libzmq.so.5
#3  0x000073729349caa4 in start_thread (arg=<optimized out>) at ./nptl/pthread_create.c:447
#4  0x0000737293529c3c in clone3 () at ../sysdeps/unix/sysv/linux/x86_64/clone3.S:78

Thread 29 (Thread 0x73728fff56c0 (LWP 30688) "ZMQbg/IO/0"):
#0  0x000073729352a042 in epoll_wait (epfd=8, events=0x73728fff41d0, maxevents=256, timeout=-1) at

In [ ]:
for obj in key_list:
    name = obj.GetName()
    classname = key.GetClassName()
    print(f"{name} --> {classname}")
    obj = key.ReadObj()
    print(f"Classe oggetto: {obj.ClassName()}")


META --> TTree
Classe oggetto: TTree
JTRIGGER::JTriggerParameters --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE_L1 --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE_L1 --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE --> TTree
Classe oggetto: TTree
KM3NET_SUMMARYSLICE --> TTree
Classe oggetto: TTree
KM3NET_EVENT --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE_L0 --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE_L2 --> TTree
Classe oggetto: TTree
KM3NET_TIMESLICE_SN --> TTree
Classe oggetto: TTree


In [7]:
tree = file.Get("KM3NET_EVENT")
tree.Print()

******************************************************************************
*Tree    :KM3NET_EVENT:                                                        *
*Entries :        6 : Total =          121018 bytes  File  Size =     118459 *
*        :          : Tree compression factor =   1.00                       *
******************************************************************************
*Branch  :KM3NET_EVENT                                                       *
*Entries :        6 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :KM3NETDAQ::JDAQPreamble : BASE                                     *
*Entries :        6 : Total  Size=        739 bytes  File Size  =        178 *
*Baskets :        1 : Basket Size=    5000000 bytes  Compression=   1.00     *
*............................................................................*
*Br    1 :KM3NETDAQ::JDAQEventHeader : BASE       

In [14]:
root -l tuofile.root

SyntaxError: invalid syntax (3199893799.py, line 1)

In [15]:
file = uproot.open(fname)
tree = file["KM3NET_EVENT"]

# Leggiamo solo il ramo dei triggered hits
arrays = tree.arrays(["triggeredHits"], library = "np")
print(arrays["triggeredHits"])

ValueError: unknown class KM3NETDAQ::JDAQTriggeredHit that cannot be skipped because its number of bytes is unknown
in file KM3NeT_00000254_00003382.root

In [9]:
tree = file.Get("KM3NET_EVENT")
tree.Print()

******************************************************************************
*Tree    :KM3NET_EVENT:                                                        *
*Entries :        6 : Total =          121018 bytes  File  Size =     118459 *
*        :          : Tree compression factor =   1.00                       *
******************************************************************************
*Branch  :KM3NET_EVENT                                                       *
*Entries :        6 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :KM3NETDAQ::JDAQPreamble : BASE                                     *
*Entries :        6 : Total  Size=        739 bytes  File Size  =        178 *
*Baskets :        1 : Basket Size=    5000000 bytes  Compression=   1.00     *
*............................................................................*
*Br    1 :KM3NETDAQ::JDAQEventHeader : BASE       